## **Preparation**
Clone our repository (https://github.com/m-depascale/AnomalySegmentation_AML/tree/main)

In [ ]:
%cd /content/
!git clone https://github.com/m-depascale/AnomalySegmentation_AML

/content
fatal: destination path 'AnomalySegmentation_AML' already exists and is not an empty directory.


In [ ]:
%cd /content/AnomalySegmentation_AML/
!git pull

/content/AnomalySegmentation_AML
Already up to date.


In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


## Some necessary module and library

In [ ]:
!pip install ood-metrics
!pip install visdom
!pip install numpy
!pip install matplotlib
!pip install Pillow
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=53d0b6acbda0e84f51cdcdfa3d2a8d889c8ea734ad77546e697bfd3d1f07d4f3
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


**Cityscape Dataset**
* Download the "leftImg8bit" for the RGB images and the "gtFine" for the labels. Please note that for training you should use the "_labelTrainIds" and not the "_labelIds, You can download the cityscapes scripts and use the conversor to generate trainIds from labelIds

In [ ]:
!python -m pip install cityscapesscripts
!python -m pip install cityscapesscripts[gui]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=cd9a2b4379e22685cd2c0eeeef560547b07be162cbaa92f3eb4254909c4e9fb7
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 9.4 MB/s eta 0:00:00


* Once you have install the scripts, you can use them to download gtFine and leftImg8bit

In [ ]:
#As shows in the README here: https://github.com/mcordts/cityscapesScripts
from cityscapesscripts.download import downloader
print(dir(downloader))
import os

# Esegui il login
session = downloader.login()
names = downloader.list_available_packages(session=session)
print(names)


destination_path = "/content/Datasets/Cityscapes"
os.makedirs(destination_path, exist_ok=True)

downloader.download_packages(session=session, package_names=['leftImg8bit_trainvaltest.zip', 'gtFine_trainvaltest.zip'], destination_path=destination_path)


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'absolute_import', 'appdirs', 'argparse', 'division', 'download_packages', 'get_available_packages', 'getpass', 'hashlib', 'input', 'json', 'list_available_packages', 'login', 'main', 'os', 'parse_arguments', 'print_function', 'requests', 'shutil', 'stat', 'unicode_literals']
Cityscapes username or email address: s314404@studenti.polito.it
Cityscapes password: ··········
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: y
The following packages are available for download.
Please refer to https://www.cityscapes-dataset.com/downloads/ for additional packages and instructions on properly citing third party packages.
 gtFine_trainvaltest.zip -> 241MB
 gtCoarse.zip -> 1.3GB
 leftImg8bit_trainvaltest.zip -> 11GB
 leftImg8bit_trainextra.zip -> 44GB
 rightImg8bit_trainvaltest.zip -> 11GB
 rightImg8bit_trainextra.zip -> 44GB
 disparity_trainval

* Now unzip the content

In [ ]:
!unzip /content/Datasets/Cityscapes/leftImg8bit_trainvaltest.zip -d /content/Datasets/Cityscapes/


In [ ]:
!unzip /content/Datasets/Cityscapes/gtFine_trainvaltest.zip -d /content/Datasets/Cityscapes/

In [ ]:
os.environ['CITYSCAPES_DATASET'] = '/content/Datasets/Cityscapes'
from cityscapesscripts.preparation.createTrainIdLabelImgs import main as convertor
convertor()

## POINT 2, DATA FOR TABLES
Execution of evalAnomaly for performing various anomaly inferences with different anomaly segmentation metrics

* retrieve the datasets from: https://drive.google.com/file/d/1r2eFANvSlcUjxcerjC8l6dRa0slowMpx/view

In [ ]:
# if the last command doesn't work try:
!pip install gdown

file_id = '1r2eFANvSlcUjxcerjC8l6dRa0slowMpx'
output_file = 'Validation_Dataset.zip'

# Scarica il file utilizzando gdown
!gdown --id $file_id -O $output_file

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1r2eFANvSlcUjxcerjC8l6dRa0slowMpx
From (redirected): https://drive.google.com/uc?id=1r2eFANvSlcUjxcerjC8l6dRa0slowMpx&confirm=t&uuid=a2628e64-2a5e-486d-838d-eec2d112a0a1
To: /content/Validation_Dataset.zip
100% 329M/329M [00:02<00:00, 161MB/s]


In [ ]:
!unzip /content/Validation_Dataset.zip -d /content

Archive:  /content/Validation_Dataset.zip
   creating: /content/Validation_Dataset/
   creating: /content/Validation_Dataset/RoadAnomaly21/
  inflating: /content/Validation_Dataset/.DS_Store  
  inflating: /content/__MACOSX/Validation_Dataset/._.DS_Store  
   creating: /content/Validation_Dataset/RoadAnomaly/
   creating: /content/Validation_Dataset/RoadObsticle21/
   creating: /content/Validation_Dataset/fs_static/
   creating: /content/Validation_Dataset/FS_LostFound_full/
  inflating: /content/Validation_Dataset/RoadAnomaly21/.DS_Store  
  inflating: /content/__MACOSX/Validation_Dataset/RoadAnomaly21/._.DS_Store  
   creating: /content/Validation_Dataset/RoadAnomaly21/images/
   creating: /content/Validation_Dataset/RoadAnomaly21/labels_masks/
  inflating: /content/Validation_Dataset/RoadAnomaly/.DS_Store  
  inflating: /content/__MACOSX/Validation_Dataset/RoadAnomaly/._.DS_Store  
   creating: /content/Validation_Dataset/RoadAnomaly/images/
   creating: /content/Validation_Dataset/

# Perform anomaly inferences on:
  - RoadAnomaly21
  - RoadObsticle21
  - Fs_static
  - fs L&F
  - Road Anomaly



In [ ]:
%cd /content/AnomalySegmentation_AML/eval/

* Commands for evalutation with Method MSP




In [ ]:
!python evalAnomaly.py --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* Commands for evalutation with MaxLogit:
  - resource: https://proceedings.mlr.press/v162/hendrycks22a/hendrycks22a.pdf



In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* Commands for evalutation with MaxEntropy
  * resource: https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark/blob/master/methods/baselines.py#L85

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* Commands for evalutation with Temperature scaling:
  * resource: : https://github.com/gpleiss/temperature_scaling

T = 0.5

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.5 --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.5 --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.5 --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.5 --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.5 --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

T = 0.75

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.75 --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.75 --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.75 --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.75 --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.75 --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

T = 1.1

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 1.1 --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 1.1 --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 1.1 --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 1.1 --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 1.1 --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

# Finding the Best Temperature
* trying range [0.1 - 2.0]

In [ ]:
import subprocess
import re
import numpy as np

AuPRC_array = []
fpr_at_95_tpr = []
AuPRC_dict = {temp: 0.0 for temp in np.arange(0.1, 2.1, 0.1)}
fpr_at_95_tpr_dict = {temp: 0.0 for temp in np.arange(0.1, 2.1, 0.1)}

dataset_paths = [
    "/content/Validation_Dataset/RoadAnomaly21/images/*.png",
    "/content/Validation_Dataset/RoadObsticle21/images/*.webp",
    "/content/Validation_Dataset/RoadAnomaly/images/*.jpg",
    "/content/Validation_Dataset/fs_static/images/*.jpg",
    "/content/Validation_Dataset/FS_LostFound_full/images/*.png",
]


for temp in np.arange(0.1, 2.1, 0.1): # finding the best temperatures from 0.1 and 2.0
  for input in dataset_paths:
    command = f"python evalAnomaly.py --method 'MSPT' --temp {temp} --input '{input}'"
    print(f"Command: {command}")
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
      print(f"Error while executing the command. Returned code: {result.returncode}")
      print(f"Error output:\n{result.stderr.decode('utf-8')}")

    output = result.stdout
    print(output)

    # Try with regular expression to retrieve decimal value
    prc_auc_match = re.search(r'AUPRC score: (\d+\.\d+)', output)
    fpr_match = re.search(r'FPR@TPR95: (\d+\.\d+)', output)

    if prc_auc_match and fpr_match:
      prc_auc = float(prc_auc_match.group(1)) # Return the string matched by the RE
      fpr = float(fpr_match.group(1))
      AuPRC_array.append((temp,prc_auc))
      fpr_at_95_tpr.append((temp, fpr))

      AuPRC_dict[temp] += prc_auc
      fpr_at_95_tpr_dict[temp] += fpr


#compute Avg for AuPRC for each temp
avg_AuPRC = [(temp, total / len(dataset_paths)) for temp, total in AuPRC_dict.items()]
print('avg_AuPRC: ', avg_AuPRC)

#compute Avg for fpr_at_95_tpr for each temp
avg_fpr_at_95_tpr = [(temp, total /len(dataset_paths)) for temp, total in fpr_at_95_tpr_dict.items()]
print('avg_fpr_at_95_tpr: ', avg_fpr_at_95_tpr)



# Avg_AuPRC:

In [ ]:
for temp, avg in avg_AuPRC:
  print('temp: ', str(temp).ljust(22),'avg AuPRC: ', str(avg))

# Avg FPR@TPR95

In [ ]:
for temp, avg in avg_fpr_at_95_tpr:
  print('temp: ', str(temp).ljust(22),'avg FPR@95TPR: ', str(avg))

# Computing the best
* first show the best w.r to AuPRC and w.r to FPR@95 on avg

In [ ]:
max_AuPRC_temp, max_AuPRC_value = max(avg_AuPRC, key=lambda x: x[1])

min_fpr_at_95_tpr_temp, min_fpr_at_95_tpr_value = min(avg_fpr_at_95_tpr, key=lambda x: x[1])

print("Temperature for retriving the max avg_AuPRC :", max_AuPRC_temp)
print("Temperature for retriving the min avg_fpr_at_95_tpr: ", min_fpr_at_95_tpr_temp)


In [ ]:
best_temp = None
best_score = float('-inf')
for temp_AuPRC, avg_AuPRC_value in avg_AuPRC:
    for temp_fpr, avg_fpr_value in avg_fpr_at_95_tpr:
        score = avg_AuPRC_value - avg_fpr_value
        if score > best_score:
            best_score = score
            best_temp = temp_AuPRC

print("Best:", best_temp)


# Perform MSP with temperature t = t_best = 0.1

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.1 --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.1 --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.1 --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.1 --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSPT' --temp 0.1 --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

# Void classifier
# Erfnet

In [ ]:
%cd /content/AnomalySegmentation_AML/trained_models

/content/AnomalySegmentation_AML/trained_models


In [ ]:
import torch

In [ ]:
state_dict_erfnet_pretrained = torch.load('erfnet_pretrained.pth')
for key, value in state_dict_erfnet_pretrained.items():
    print(key)

module.encoder.initial_block.conv.weight
module.encoder.initial_block.conv.bias
module.encoder.initial_block.bn.weight
module.encoder.initial_block.bn.bias
module.encoder.initial_block.bn.running_mean
module.encoder.initial_block.bn.running_var
module.encoder.layers.0.conv.weight
module.encoder.layers.0.conv.bias
module.encoder.layers.0.bn.weight
module.encoder.layers.0.bn.bias
module.encoder.layers.0.bn.running_mean
module.encoder.layers.0.bn.running_var
module.encoder.layers.1.conv3x1_1.weight
module.encoder.layers.1.conv3x1_1.bias
module.encoder.layers.1.conv1x3_1.weight
module.encoder.layers.1.conv1x3_1.bias
module.encoder.layers.1.conv3x1_2.weight
module.encoder.layers.1.conv3x1_2.bias
module.encoder.layers.1.conv1x3_2.weight
module.encoder.layers.1.conv1x3_2.bias
module.encoder.layers.1.bn1.weight
module.encoder.layers.1.bn1.bias
module.encoder.layers.1.bn1.running_mean
module.encoder.layers.1.bn1.running_var
module.encoder.layers.1.bn2.weight
module.encoder.layers.1.bn2.bias
mod

In [ ]:
state_dict_erfnet_finetuned = torch.load('erfnet_finetuned_on_20_classes.pth')
for key, value in state_dict_erfnet_finetuned.items():
    print(key)

encoder.initial_block.conv.weight
encoder.initial_block.conv.bias
encoder.initial_block.bn.weight
encoder.initial_block.bn.bias
encoder.initial_block.bn.running_mean
encoder.initial_block.bn.running_var
encoder.initial_block.bn.num_batches_tracked
encoder.layers.0.conv.weight
encoder.layers.0.conv.bias
encoder.layers.0.bn.weight
encoder.layers.0.bn.bias
encoder.layers.0.bn.running_mean
encoder.layers.0.bn.running_var
encoder.layers.0.bn.num_batches_tracked
encoder.layers.1.conv3x1_1.weight
encoder.layers.1.conv3x1_1.bias
encoder.layers.1.conv1x3_1.weight
encoder.layers.1.conv1x3_1.bias
encoder.layers.1.bn1.weight
encoder.layers.1.bn1.bias
encoder.layers.1.bn1.running_mean
encoder.layers.1.bn1.running_var
encoder.layers.1.bn1.num_batches_tracked
encoder.layers.1.conv3x1_2.weight
encoder.layers.1.conv3x1_2.bias
encoder.layers.1.conv1x3_2.weight
encoder.layers.1.conv1x3_2.bias
encoder.layers.1.bn2.weight
encoder.layers.1.bn2.bias
encoder.layers.1.bn2.running_mean
encoder.layers.1.bn2.runn

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

In [ ]:
!python evalAnomaly.py --method 'VOID' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg" --loadWeights 'erfnet_finetuned_on_20_classes.pth'

In [ ]:
!python evalAnomaly.py --method 'VOID' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png" --loadWeights 'erfnet_finetuned_on_20_classes.pth'

In [ ]:
!python evalAnomaly.py --method 'VOID' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp" --loadWeights 'erfnet_finetuned_on_20_classes.pth'

In [ ]:
!python evalAnomaly.py --method 'VOID' --input "/content/Validation_Dataset/fs_static/images/*.jpg" --loadWeights 'erfnet_finetuned_on_20_classes.pth'

In [ ]:
!python evalAnomaly.py --method 'VOID' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png" --loadWeights 'erfnet_finetuned_on_20_classes.pth'

## Enet

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'ENet_pretrained' --loadModel 'enet.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'ENet_pretrained' --loadModel 'enet.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'ENet_pretrained' --loadModel 'enet.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'ENet_pretrained' --loadModel 'enet.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'ENet_pretrained' --loadModel 'enet.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

## BiSeNet

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuning_final.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuning_final.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuning_final.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuning_final.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuning_final.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

# Compute mIoU for each model

In [ ]:
!pwd
%cd /content/AnomalySegmentation_AML/eval

## mIoU on Val dataset

* Erfnet trained on 19 classes

In [ ]:
!python eval_iou.py --datadir "/content/Datasets/Cityscapes/" --subset 'val'  # by default '--loadWeights' = 'erfnet_pretrained.pth' and '--loadModel' = 'erfnet.py'

* Erfnet finetuned on 20 classes

In [ ]:
!python eval_iou.py --datadir "/content/Datasets/Cityscapes/" --subset 'val' --loadWeights='erfnet_finetuned_on_20_classes.pth' # by default  and '--loadModel' = 'erfnet.py'

* Enet

In [ ]:
!python eval_iou.py --datadir "/content/Datasets/Cityscapes/" --subset 'val' --loadWeights='ENet_pretrained' --loadModel='enet.py'

* BiSeNet

In [ ]:
!python eval_iou.py --datadir "/content/Datasets/Cityscapes/" --subset 'val' --loadWeights='bisenet_finetuning_final.pth' --loadModel='bisenetv1.py'

# Anomaly inferences on Bisenet Pruned

* Void

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MSP

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxLogit

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxEntropy

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

# Anomaly inferences on Channel-Based ERFNet Pruned

* Void

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'VOID' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MSP

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxLogit

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxEntropy

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'erfnet_pruned_all_2.pth' --loadModel 'channel_based_erfnet_pruned.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

# Anomaly  inferences on original Bisenet

* MSP

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MSP' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxLogit

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxLogit' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

* MaxEntropy

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

In [ ]:
!python evalAnomaly.py --method 'MaxEntropy' --loadWeights 'bisenet_finetuned_after_pruning.pth' --loadModel 'bisenetv1.py' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"